### Goal

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd

# to prevent figure 
# to show in the notebook
import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt

import plotting as plotting

### TODO: update this

In [3]:
date = "24-12-15"
pathfile = f"Pulling data/{date}/single/"

### From here; automatic run

In [4]:
root = f"/Users/michaelmoret/Library/CloudStorage/GoogleDrive-michael@externa.bio/.shortcut-targets-by-id/1BdUNsBjDh5Gee_76jCiKB1C_CwG0ercP/{pathfile}"

MODE = "single"
dev = False

In [5]:
if not root.endswith("/"):
    root += "/"

sep_category = "$" # separates categories like date, experiments, etc
sep_count = "%" # separated the number of hairs by experiment

In [6]:
all_dfs = []
all_fns = []

for file in os.listdir(root):
    if file.endswith(".txt"):
        print("\n*****************************")
        print(file)
        df = plotting.get_df_from_file(root + file,
                                       skip=10)
        # clean the df
        if MODE in file:
            df = plotting.clean_single(df)
            assert "single" in file
        else:
            raise ValueError(f"Not a {MODE} experiment")

        splitted_name = file.split(sep_category)
        # get the experiments; i.e. not the date
        # not the single or triple etc
        splitted_name = splitted_name[2:-1]
        print("splitted_name: ", splitted_name)
        # add the experiment name in the dataframe
        all_names = []
        for entries in splitted_name:
            times_name = entries.split(sep_count)
            _times = int(times_name[0])
            _name = times_name[1]
            all_names += [_name] * _times
        print(f"len df: {len(df)}, len names: {len(all_names)}")
        df["Name"] = all_names
        # remove nan
        df = df.dropna()
        all_dfs.append(df)
        all_fns.append(file.replace(".txt", ""))


*****************************
20241214$IHNY$25%SDS$25%PB 4x$25%ELI 2-4-K 0.03 4x$single.txt
Missing records: []
splitted_name:  ['25%SDS', '25%PB 4x', '25%ELI 2-4-K 0.03 4x']
len df: 75, len names: 75

*****************************
20241214$IHNY$25%PB 8x$26%ELI 2-4-k 0.03 8x$single.txt
Missing records: []
splitted_name:  ['25%PB 8x', '26%ELI 2-4-k 0.03 8x']
len df: 51, len names: 51


In [7]:
def master_plots(df, header, ymin, ymax, savepath, show=False, verbose=False):
    fig, ax, removed = plotting.create_boxplot(df, header, ymin, ymax)
    #save removed data
    removed.to_csv(f"{savepath}{header}_removed.csv", index=True)
    if verbose:
        print(f"{len(removed)} removed data points for {header}")
    # plot
    fig.savefig(f"{savepath}{header}.png")
    if show: 
        plt.show()
        plt.close()

In [8]:
len(all_dfs), len(all_fns)

(2, 2)

In [9]:
all_fns

['20241214$IHNY$25%SDS$25%PB 4x$25%ELI 2-4-K 0.03 4x$single',
 '20241214$IHNY$25%PB 8x$26%ELI 2-4-k 0.03 8x$single']

In [10]:
for df, name in zip(all_dfs, all_fns):
    print("\n************************************")
    print(f"{name}\n")
    print(f"df length: {len(df)}")
    try:
        savepath = f"{root}{name}/"
        if dev:
            savepath = f"{root}dev/"
        os.makedirs(savepath, exist_ok=True)
        
        try:
            header = 'TENSILE_STRENGTH'
            ymin = 90
            ymax = 325
            master_plots(df, header, ymin, ymax, savepath)
        except Exception as e:
            print(f"ERROR with {header}")
            print("Error:", e)

        try:
            header = 'BREAK_STRAIN(*)(#)'
            ymin = 0
            ymax = 100
            master_plots(df, header, ymin, ymax, savepath)
        except Exception as e:
            print(f"ERROR with {header}")
            print("Error:", e)

        try:
            header = 'BREAK_LOAD'
            ymin = 0
            ymax = 2.0
            master_plots(df, header, ymin, ymax, savepath)
        except Exception as e:
            print(f"ERROR with {header}")
            print("Error:", e)
        
        try:
            header = 'EMOD(*)(#)'
            ymin = 2.5
            ymax = 8.0
            master_plots(df, header, ymin, ymax, savepath)
        except Exception as e:
            print(f"ERROR with {header}")
            print("Error:", e)

        # scatter plots
        y_col = 'TENSILE_STRENGTH'
        x_col = 'MEAN AREA'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

        y_col = 'EMOD(*)(#)'
        x_col = 'MEAN AREA'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

        y_col = 'BREAK_LOAD'
        x_col = 'MEAN AREA'
        savedir = f"{savepath}correlation_plot/"
        plt = plotting.create_scatter_plot(df, x_col, y_col, savedir)

    except:
        print(f"\nERROR with {name}\n")
        continue

print("\n\nplotting DONE")


************************************
20241214$IHNY$25%SDS$25%PB 4x$25%ELI 2-4-K 0.03 4x$single

df length: 70

Creating boxplot for TENSILE_STRENGTH
Removed outliers for SDS : 24    152.6
Name: TENSILE_STRENGTH, dtype: float64
Removed outliers for PB 4x : 28    150.9
35    167.2
Name: TENSILE_STRENGTH, dtype: float64
Removed outliers for ELI 2-4-K 0.03 4x : Series([], Name: TENSILE_STRENGTH, dtype: float64)
Median for SDS: 211.1
Median for PB 4x: 214.8
Median for ELI 2-4-K 0.03 4x: 206.0

Creating boxplot for BREAK_STRAIN(*)(#)
Removed outliers for SDS : 24    28.357
Name: BREAK_STRAIN(*)(#), dtype: float64
Removed outliers for PB 4x : 28    26.608
Name: BREAK_STRAIN(*)(#), dtype: float64
Removed outliers for ELI 2-4-K 0.03 4x : Series([], Name: BREAK_STRAIN(*)(#), dtype: float64)
Stats. significance: SDS and PB 4x: p = 0.0006
Stats. significance: SDS and ELI 2-4-K 0.03 4x: p = 0.0265
Median for SDS: 46.656
Median for PB 4x: 52.026
Median for ELI 2-4-K 0.03 4x: 51.065

Creating boxplo